In [2]:
# Forked from Josh

import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
import pylab


In [3]:
nc_file = 'Geospatial_Height.nc'
file = Dataset(nc_file, mode='r')

In [4]:
lons = file.variables['X'][:]
lats = file.variables['Y'][:]
pressure = file.variables['P'][:]
time = file.variables['T'][:]
height = file.variables['phi'][:]

In [5]:
file.close()

In [6]:
data = height.reshape(24886, 4464).astype(np.float64)

In [7]:
data

array([[ 5146.,  5145.,  5143., ...,  5829.,  5829.,  5830.],
       [ 5103.,  5105.,  5107., ...,  5834.,  5834.,  5836.],
       [ 5067.,  5068.,  5067., ...,  5840.,  5839.,  5839.],
       ..., 
       [ 5210.,  5210.,  5209., ...,  5883.,  5885.,  5886.],
       [ 5138.,  5149.,  5161., ...,  5886.,  5888.,  5888.],
       [ 5139.,  5155.,  5172., ...,  5887.,  5888.,  5889.]])

In [8]:
# http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html
# Center to the mean and component wise scale to unit variance.
scaled_data = scale(data)

In [9]:
# http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
pca = PCA(n_components=10)
pca.fit(scaled_data)

# transform
data_reduced = np.dot(scaled_data, pca.components_.T)
# inverse_transform
data_original = np.dot(data_reduced, pca.components_)

In [10]:
print np.shape(data_original)

(24886L, 4464L)


In [11]:
pca.explained_variance_ratio_

array([ 0.44734135,  0.14372132,  0.03143608,  0.02327541,  0.02048559,
        0.01685406,  0.01567907,  0.01401042,  0.01340728,  0.01317348])

In [12]:
n_components = len(pca.explained_variance_ratio_)
total_variance = sum(pca.explained_variance_ratio_)
print("The first {0} components explain {1}% of the variance".format(n_components, total_variance * 100))

The first 10 components explain 73.9384065597% of the variance


In [13]:
# These are eigenvectors!
pca.components_

array([[ 0.01655331,  0.01649645,  0.01645068, ...,  0.0091155 ,
         0.00905843,  0.00902507],
       [ 0.00428184,  0.00440805,  0.0045381 , ..., -0.0304706 ,
        -0.03054783, -0.03064186],
       [ 0.0107967 ,  0.01024009,  0.00972206, ...,  0.01433512,
         0.01424309,  0.01390165],
       ..., 
       [ 0.00923061,  0.00824647,  0.00709655, ..., -0.00413104,
        -0.00334936, -0.00267937],
       [-0.01219722, -0.01188298, -0.01157935, ...,  0.01477611,
         0.01514039,  0.01543892],
       [ 0.01783526,  0.0186562 ,  0.0194143 , ...,  0.01723133,
         0.01736719,  0.01744784]])

In [14]:
# Each day has been reduced to have only 10 values
data_reduced.shape

(24886L, 10L)

In [15]:
# recover the original shape of the data
recovered_data = data_original.reshape(24886, 1, 31, 144)

In [16]:
# for example, this is January 1, 1948 as reconstructed by the 10 principal components
recovered_data[0, 0].shape

(31L, 144L)

In [17]:
# Let's plot some stuff
m = Basemap(projection='robin', lat_0=0, lon_0=-100,
              resolution='l', area_thresh=1000.0)

# Because our lon and lat variables are 1D, 
# use meshgrid to create 2D arrays 
# Not necessary if coordinates are already in 2D arrays.
lon, lat = np.meshgrid(lons, lats)
xi, yi = m(lon, lat)

In [18]:
def plot_day(data, day):
    """
    Creates a plot of the world with colorcoded data
    """
    plt.figure(figsize=(15,12))
    # Plot Data
    cs = m.pcolor(xi,yi,data[day, 0])

    # Add Grid Lines
    m.drawparallels(np.arange(-80., 81., 10.), labels=[1,0,0,0], fontsize=10)
    m.drawmeridians(np.arange(-180., 181., 10.), fontsize=10)

    # Add Coastlines, States, and Country Boundaries
    m.drawcoastlines()
    m.drawstates()
    m.drawcountries()
    # Shaded relief adds the nice global color
    m.shadedrelief()

    # Add Colorbar
    cbar = m.colorbar(cs, location='bottom', pad="10%")

    # Add Title
    plt.title("Reconstructed Geopotential Height {} Days Since 1-1-1948".format(day))

In [21]:
# This can be slow the first time you all it
# plot_day(recovered_data, 23675)
plotdata = data.reshape(24886, 1, 31, 144)
plot_day(plotdata, 23673)

In [22]:
pylab.show()

In [57]:
# PCA data for only NJ 
# 16 corresponds to latitude 40 and 102 corresponds to longitude 74.5
# Since the precipiations is available at monthly level - we will take one value of PCA every 30 days - total of 816 months so pick 1 day every 30 days

PCA_list = []
for i in range(804):
    PCA_list.append(np.float(recovered_data[0+i*30,:,16,102]))
    
print np.shape(PCA_list)
                    

(804L,)


In [65]:
from numpy import genfromtxt
# http://climate.rutgers.edu/stateclim_v1/data/njhistprecip.html
my_data = genfromtxt('New Jersey Flood Data.csv', delimiter=',')

rain_list = []
for i in range(67):
    for j in range(12):
        rain_list.append(my_data.item(i,j))
        
print np.shape(rain_list)

(804L,)


In [91]:
from pylab import *
import matplotlib.pyplot as plt
import scipy
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(PCA_list, rain_list)

plot = plt.scatter(PCA_list,rain_list)
plt.title("Rainfall v. PC1 in New Jersey")
plt.xlabel('PC1')
plt.ylabel('Rainfall (inches)')
# plt.show()

print slope, intercept, r_value, p_value, std_err

 0.0238990174006 3.87791278063 0.0110185283191 0.755076561996 0.0765848567072
